![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

## Data aquisition

In [1]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from helper import cumulated_returns, calculate_return, calculate_percentage_return, calculate_return_in_usd, calculate_return_in_mxn, calculate_transaction_costs, expiration_dates, calculate_and_merge_returns
import numpy as np
from scipy.stats import skew

In [ ]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'expiration_dates': [3, 6, 9, 12]
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,
        'commission': 2.50,
        'expiration_dates': [3, 6, 9, 12]
    },
    'ZN': {  # 10-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.015625,
        'minimum_fluctuation': 15.625,
        'spread': 0.015625,
        'commission': 2.50,
        'expiration_dates': [6, 9, 12]  # Placeholder
    },
    'ZF': {  # 5-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.0078125,
        'minimum_fluctuation': 7.8125,
        'spread': 0.0078125,
        'commission': 2.50,
        'expiration_dates': [6, 9, 12]  # Placeholder
    },
    'ZB': {  # 30-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.015625,
        'minimum_fluctuation': 15.625,
        'spread': 0.015625,
        'commission': 2.50,
        'expiration_dates': [6, 9, 12]  # Placeholder
    },
    'ZW': {  # Wheat
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'CL': {  # Crude Oil
        'multiplier': 1000,
        'tick_value': 0.01,
        'minimum_fluctuation': 10.00,
        'spread': 0.01,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'GC': {  # Gold
        'multiplier': 100,
        'tick_value': 0.1,
        'minimum_fluctuation': 10.00,
        'spread': 0.1,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'HE': {  # Lean Hog
        'multiplier': 400,
        'tick_value': 0.00025,
        'minimum_fluctuation': 10.00,
        'spread': 0.00025,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'KC': {  # Coffee
        'multiplier': 375,
        'tick_value': 0.0005,
        'minimum_fluctuation': 18.75,
        'spread': 0.0005,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
        'SB': {  # Sugar
        'multiplier': 1120,
        'tick_value': 0.0001,
        'minimum_fluctuation': 11.20,
        'spread': 0.0001,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'GF': {  # Feeder Cattle
        'multiplier': 500,
        'tick_value': 0.00025,
        'minimum_fluctuation': 12.50,
        'spread': 0.00025,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'ZS': {  # Soy Bean
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'ZC': {  # Corn
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'NG': {  # Natural Gas
        'multiplier': 10000,
        'tick_value': 0.001,
        'minimum_fluctuation': 10.00,
        'spread': 0.001,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'SI': {  # Silver
        'multiplier': 5000,
        'tick_value': 0.005,
        'minimum_fluctuation': 25.00,
        'spread': 0.005,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'HG': {  # Copper
        'multiplier': 25000,
        'tick_value': 0.0005,
        'minimum_fluctuation': 12.50,
        'spread': 0.0005,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'NQ': {  # Nasdaq
        'multiplier': 20,
        'tick_value': 0.25,
        'minimum_fluctuation': 5.00,
        'spread': 0.25,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    },
    'CT': {  # Cotton
        'multiplier': 500,
        'tick_value': 0.0001,
        'minimum_fluctuation': 5.00,
        'spread': 0.0001,
        'commission': 2.50,
        'expiration_dates': []  # Placeholder
    }
}



In [ ]:
# Print each symbol in instruments dict
for symbol in instruments:
    print(symbol)

In [2]:
# Initialize QuantBook
qb = QuantBook()

# Add Future Subscriptions for ES and MES with Daily resolution
es_future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
mes_future = qb.AddFuture(Futures.Indices.MicroSP500EMini, Resolution.Daily)

# Add Forex Subscription for USD/MXN with Daily resolution
usdmxn = qb.AddForex("USDMXN").symbol

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data for ES and MES futures
es_history = qb.History([es_future.Symbol], start_date, end_date, Resolution.Daily)
mes_history = qb.History([mes_future.Symbol], start_date, end_date, Resolution.Daily)

# Request Historical Data for USD/MXN FX pair
fx_history = qb.History([usdmxn], start_date, end_date, Resolution.Daily)

# Drop contract name from multi-index
es_history.reset_index(level=0, drop=True, inplace=True)
mes_history.reset_index(level=0, drop=True, inplace=True)

trading_days_per_year = 256

# Print names of the constructed variables in the cell

print ("es_history: DataFrame")
print ("mes_history: DataFrame")
print ("fx_history: DataFrame")
print ("trading_days_per_year: scalar")
print ("start_date and end_date: datetime objects")



es_history: DataFrame
mes_history: DataFrame
fx_history: DataFrame
trading_days_per_year: scalar
start_date and end_date: datetime objects


In [ ]:
print(es_history.index)
print(mes_history.index)
print(fx_history.index)


## Instrument dictionary

Each contract may have different multipliers. a multiplier of 50 means that a 1$ move is equal to a 50$ change in returns for each contract one is exposed to. We will create a dictionary with the relevant information to calculate the returns down the road.

### Compute Functions

In [3]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['ES']
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['MES']
    }
}
for key in instruments:
    print(key)
    print(instruments[key])

ES
{'multiplier': 50, 'tick_value': 0.25, 'minimum_fluctuation': 12.5, 'spread': 0.25, 'commission': 2.5, 'expiration_dates': [datetime.datetime(2007, 3, 16, 0, 0), datetime.datetime(2007, 6, 15, 0, 0), datetime.datetime(2007, 9, 21, 0, 0), datetime.datetime(2007, 12, 21, 0, 0), datetime.datetime(2008, 3, 21, 0, 0), datetime.datetime(2008, 6, 20, 0, 0), datetime.datetime(2008, 9, 19, 0, 0), datetime.datetime(2008, 12, 19, 0, 0), datetime.datetime(2009, 3, 20, 0, 0), datetime.datetime(2009, 6, 19, 0, 0), datetime.datetime(2009, 9, 18, 0, 0), datetime.datetime(2009, 12, 18, 0, 0), datetime.datetime(2010, 3, 19, 0, 0), datetime.datetime(2010, 6, 18, 0, 0), datetime.datetime(2010, 9, 17, 0, 0), datetime.datetime(2010, 12, 17, 0, 0), datetime.datetime(2011, 3, 18, 0, 0), datetime.datetime(2011, 6, 17, 0, 0), datetime.datetime(2011, 9, 16, 0, 0), datetime.datetime(2011, 12, 16, 0, 0), datetime.datetime(2012, 3, 16, 0, 0), datetime.datetime(2012, 6, 15, 0, 0), datetime.datetime(2012, 9, 21, 0

## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns Time Series

In [4]:
# Definir una función para calcular y unir retornos para un símbolo dado
def calculate_and_merge_returns(symbol, price_series, instruments):
    """
    Inputs 
    symbol: str - a symbol to calculate returns for
    price_series: pd.Series - a price series to calculate returns from
    instruments: dict - a dictionary of instruments with metadata

    Returns
    merged_df: pd.DataFrame - a DataFrame with both return and percentage return columns
    """
    returns_df = calculate_return(symbol, price_series, instruments)
    percentage_returns = calculate_percentage_return(symbol, price_series, instruments, num_contracts=1)
    
    # Unir los DataFrames en base al índice
    merged_df = returns_df.join(percentage_returns.rename('percentage_return'))
    # Clean up the memory
    del returns_df
    del percentage_returns
    return merged_df


In [8]:
# Create a dictionary to store timeseries data for each symbol
timeseries_returns = {}

# Calculate and store the returns data in separate DataFrames
es_returns = calculate_and_merge_returns('ES', es_history['close'], instruments)
mes_returns = calculate_and_merge_returns('MES', mes_history['close'], instruments)

# Store the DataFrames in the dictionary
timeseries_returns['ES'] = es_returns
timeseries_returns['MES'] = mes_returns


### Mean Returns for each symbol

In [7]:
# Create a dictionary to store the summary statistics
summary = {}

# Iterate over each symbol in the timeseries_returns dictionary
for symbol, df in timeseries_returns.items():
    # Calculate the mean of the currency returns
    mean_return = df['currency_return'].mean()
    # Calculate the standard deviation of the currency returns
    std_return = df['currency_return'].std()
    # Store the results in the summary dictionary
    summary[symbol] = {
        'mean_return': mean_return,
        'std_return': std_return
    }

# Print the summary statistics for each symbol
print("Summary Statistics for Each Symbol:")
for symbol, stats in summary.items():
    print(f"{symbol} - Mean Return: {stats['mean_return']}, Standard Deviation: {stats['std_return']}")

# Clean memory
del df


Summary Statistics for Each Symbol:
ES - Mean Return: 44.210660964718315, Standard Deviation: 1480.4298643040202
MES - Mean Return: 8.786321385477473, Standard Deviation: 238.65538784453912


In [9]:

import pandas as pd
import numpy as np
from scipy.stats import skew

# Create a dictionary to store the summary statistics
summary = {}

# Iterate over each symbol in the timeseries_returns dictionary
for symbol, df in timeseries_returns.items():
    # Calculate the mean of the percentage returns
    mean_return = df['percentage_return'].mean()
    # Calculate the standard deviation of the percentage returns
    std_dev = df['percentage_return'].std()
    # Annualize the mean return and standard deviation
    annualized_mean = mean_return * trading_days_per_year
    annualized_std_dev = std_dev * (trading_days_per_year ** 0.5)
    # Calculate the daily Sharpe ratio (assuming risk-free rate is 0)
    daily_sharpe_ratio = mean_return / std_dev
    # Calculate the annualized Sharpe ratio
    annualized_sharpe_ratio = daily_sharpe_ratio * (trading_days_per_year ** 0.5)
    
    # Resample to monthly returns and calculate skewness
    df = df.reset_index(level='symbol', drop=True)
    monthly_returns = df['percentage_return'].resample('M').apply(lambda x: ((x + 100).prod() - 1) * 100)
    monthly_skew = skew(monthly_returns.dropna())

    # Calculate lower and higher fat tail ratios
    demeaned_returns = df['percentage_return'] - mean_return
    p1 = np.percentile(demeaned_returns, 1)
    p30 = np.percentile(demeaned_returns, 30)
    p70 = np.percentile(demeaned_returns, 70)
    p99 = np.percentile(demeaned_returns, 99)
    
    lpr = p1 / p30
    upr = p99 / p70
    lower_fat_tail_ratio = lpr / 4.43
    higher_fat_tail_ratio = upr / 4.43
    
    # Store the results in the summary dictionary
    summary[symbol] = {
        'mean_return': mean_return,
        'std_dev': std_dev,
        'annualized_mean': annualized_mean,
        'annualized_std_dev': annualized_std_dev,
        'daily_sharpe_ratio': daily_sharpe_ratio,
        'annualized_sharpe_ratio': annualized_sharpe_ratio,
        'monthly_skew': monthly_skew,
        'lower_fat_tail_ratio': lower_fat_tail_ratio,
        'higher_fat_tail_ratio': higher_fat_tail_ratio
    }

# Print the summary statistics for each symbol
print("Summary Statistics for Each Symbol:")
for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(f"  Mean Return: {stats['mean_return']}")
    print(f"  Standard Deviation: {stats['std_dev']}")
    print(f"  Annualized Mean: {stats['annualized_mean']}")
    print(f"  Annualized Standard Deviation: {stats['annualized_std_dev']}")
    print(f"  Daily Sharpe Ratio: {stats['daily_sharpe_ratio']}")
    print(f"  Annualized Sharpe Ratio: {stats['annualized_sharpe_ratio']}")
    print(f"  Monthly Skew: {stats['monthly_skew']}")
    print(f"  Lower Fat Tail Ratio: {stats['lower_fat_tail_ratio']}")
    print(f"  Higher Fat Tail Ratio: {stats['higher_fat_tail_ratio']}")

del df


Summary Statistics for Each Symbol:
ES:
  Mean Return: 0.03807680496751644
  Standard Deviation: 1.250701025714672
  Annualized Mean: 9.747662071684209
  Annualized Standard Deviation: 20.011216411434752
  Daily Sharpe Ratio: 0.030444370144942275
  Annualized Sharpe Ratio: 0.4871099223190764
  Monthly Skew: 1.9369922476786035
  Lower Fat Tail Ratio: 2.5655756580789677
  Higher Fat Tail Ratio: 1.7480680421214856
MES:
  Mean Return: 0.05157350417429296
  Standard Deviation: 1.3027125939212676
  Annualized Mean: 13.202817068618998
  Annualized Standard Deviation: 20.84340150274028
  Daily Sharpe Ratio: 0.039589318791378725
  Annualized Sharpe Ratio: 0.6334291006620596
  Monthly Skew: 1.9917073343576628
  Lower Fat Tail Ratio: 2.1236774147133075
  Higher Fat Tail Ratio: 1.4232109587283057


## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [21]:

# Definir la función de simulación
def simulate_buy_and_hold(history_df, symbol, fx_rates_df, num_contracts=1):
    params = instruments[symbol]
    initial_price = history_df['close'].iloc[0]
    final_price = history_df['close'].iloc[-1]

    total_return_in_usd = calculate_return_in_usd(symbol, initial_price, final_price, instruments)
    total_return_in_mxn = calculate_return_in_mxn(total_return_in_usd, fx_rates_df)

    # Calcular costos de transacción
    transaction_costs = calculate_transaction_costs(params, history_df, start_date, end_date)

    # Ajustar retornos por costos de transacción
    net_return_in_usd = total_return_in_usd - transaction_costs
    net_return_in_mxn = calculate_return_in_mxn(net_return_in_usd, fx_rates_df)

    # Multiplicar los retornos por el número de contratos
    net_return_in_usd *= num_contracts
    net_return_in_mxn *= num_contracts

    return {
        "return_in_points": (final_price - initial_price) * num_contracts,
        "return_in_usd": net_return_in_usd,
        "return_in_mxn": net_return_in_mxn
    }

In [22]:

# Llamar a la función y asignar los valores retornados a variables
num_contracts = 1  # Ejemplo: se pueden ajustar los contratos según sea necesario
es_buyhold = simulate_buy_and_hold(es_history, symbol, fx_history, num_contracts)

# Asignar los valores retornados a variables específicas
es_buyhold_points = es_buyhold["return_in_points"]
es_buyhold_usd = es_buyhold["return_in_usd"]
es_buyhold_mxn = es_buyhold["return_in_mxn"]

# Imprimir fechas
print(f"Start Date: {es_history.index[0]}")
print(f"End Date: {es_history.index[-1]}")
# Imprimir los resultados
print(f"Return in Points: {es_buyhold_points}")
print(f"Return in USD: {es_buyhold_usd}")
print(f"Return in MXN: {es_buyhold_mxn}")


In [10]:
print (df.index)

# Strategy 2: Risk measurement, risk target and capital
